In [ ]:
import numpy as np

from matplotlib import pyplot as plt

import sympy

Might need to do some analytic calculations...

In [ ]:
oo = sympy.oo

In [ ]:
R, Ie, Re, n, bn, R1 = sympy.symbols('R, I_e, R_e, n, b_n, R_1', real=True, positive=True)

In [ ]:
sersic = Ie * sympy.exp(-bn*((R/Re)**(1/n)-1))
sersic

Start by plotting the sersic fomula up to scope out where the integral might be tractible:

In [ ]:
ICOEFFS = [2, -1/3, 4/405, 46/25515, 131/1148175, -2194697/30690717750]
def f_bn(n):
    """
    Ciotti and Bertin 99, valid for n >~ 0.36
    """
    return np.sum([C*n**(1-i) for i, C in enumerate(ICOEFFS)], axis=0)

def sersic_profile(R, Ie, Re, n):
    bn = f_bn(n)
    return Ie * np.exp(-bn*((R/Re)**(1/n)-1))

def log_sersic_profile(R, Ie, Re, n):
    bn = f_bn(n)
    return np.log(Ie) - bn*((R/Re)**(1/n)-1)

In [ ]:
r = np.linspace(0, 3, 1025)[1:]
for n in [.5, 1, 2, 3, 4]:
    plt.plot(r, sersic_profile(r, 1, 1, n), label=n)
plt.legend(loc=0)
plt.axvline(1, c= 'k', ls=':')

In [ ]:
fig, axs = plt.subplots(3, 2, figsize=(12, 12))
r = np.linspace(0, 10, 1025)[1:]
for ax, n in zip(axs.ravel(), [.5, 1, 2, 4, 6, 8]):
    ax.semilogx(r, sersic_profile(r, 1, 1, n))
    ax.set_title(f'$n={n}$')
    ax.axvline(1, c= 'k', ls=':')

Now lets see how the integrand behaves since what we really need is to integrate R*I dr:

In [ ]:
r = np.linspace(0, 5, 1025)[1:]
for n in [.5, 1, 2, 3, 4]:
    plt.plot(r, r*sersic_profile(r, 1, 1, n), label=n)
plt.legend(loc=0)
plt.axvline(1, c= 'k', ls=':')

In [ ]:
fig, axs = plt.subplots(3, 2, figsize=(12, 12))
r = np.logspace(-6, 3, 1024*4 + 1)[1:]
for ax, n in zip(axs.ravel(), [.5, 1, 2, 4, 6, 8]):
    ax.semilogx(r, r*sersic_profile(r, 1, 1, n))
    ax.set_title(f'$n={n}$')
    ax.axvline(1, c= 'k', ls=':')

Ok, so the integrand is dominated by the peak and out to ~0.4-2 orders of magnitude above/below it

Can we analytically work out the peak by solving where the derivative is 0?

In [ ]:
solns = sympy.solve(sympy.diff(R*sersic, R), R)
assert len(solns)==1
solns[0]

Well that's shockingly straightfoward...

In [ ]:
def integral_peak(Re, n):
    bn = f_bn(n)
    return Re*(n/bn)**n

test_ns = np.array([.5, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
peaks = integral_peak(1, test_ns)

fig, (ax1, ax2) = plt.subplots(2, 1)
ax1.plot(test_ns, peaks, '-o')
ax2.semilogy(test_ns, peaks, '-o')

Analytical solutions for the circular integral?

In [ ]:
sympy.integrate(R*sersic, (R,0, oo))

In [ ]:
sympy.integrate(R*sersic, (R,0, R1))

Do some timing tests to see at what data size the analytical solution is ~= direct summation: